In [1]:
import bs4
import time
import requests
import pandas as pd
from bs4 import BeautifulSoup 
from collections import defaultdict

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
'''
The data required for crawling is saved in a dynamic table that updates with more rows everytime the page is
scrolled to the bottom, this script will scroll to the bottom of the page revealing all data.
'''

def scrollPage(URL):
    driver = webdriver.Chrome(ChromeDriverManager().install())
    driver.get(URL_test)
    time.sleep(10)                       # Wait for page to load

    last_height = driver.execute_script('return document.querySelector("table").scrollHeight')
    element = driver.find_element(By.TAG_NAME, "table")

    while True:
        element.send_keys(Keys.END)      # Scroll to end of available page
        time.sleep(3)                    # Wait for new content to load

        new_height = driver.execute_script('return document.querySelector("table").scrollHeight')
        if new_height == last_height:    # End of page
             break
        last_height = new_height

    return driver.page_source            # html

In [3]:
'''
After all the page is loaded, this script will crawl the page and retrieve the data.
'''

def crawlHTML(htmlText):    
    soup = BeautifulSoup(htmlText)
    titles, columns = {}, defaultdict(list)
    i = 0
    
    # Save collumn number of each header for later access
    for t in soup.findAll("th"):         
        titles[t.find("span").string.strip()] = i
        i += 1
    
    # Create a dictionary of {header title : list of collumn values}
    for title in titles:
        for value in soup.findAll("td", attrs={"data-col":str(titles[title])}):
            columns[title].append(value.text)
    
    return pd.DataFrame(columns)         # DataFrame

In [4]:
def saveDataFrameToCSV(df, path):
    df.to_csv(path, index=False)

In [5]:
'''
Driver Code
'''

# URL:       Page with all wildfire records since 2014, USA only
# URL_test:  Page with all ongoing wildfires, USA only, short table

URL = "https://data-nifc.opendata.arcgis.com/datasets/wfigs-wildland-fire-locations-full-history/explore?location=0.000000%2C0.000000%2C0.00&showTable=true"
URL_test = "https://data-nifc.opendata.arcgis.com/datasets/wfigs-current-wildland-fire-locations/explore?location=-0.000000%2C0.000000%2C7.50&showTable=true"
path_csv = "Wildland Fires.csv"

htmlText = scrollPage(URL_test)
df = crawlHTML(htmlText)
saveDataFrameToCSV(df, path_csv)

print("Task complete")



====== WebDriver manager ======
Current google-chrome version is 96.0.4664
Get LATEST chromedriver version for 96.0.4664 google-chrome
Driver [C:\Users\tspie\.wdm\drivers\chromedriver\win32\96.0.4664.45\chromedriver.exe] found in cache
<ipython-input-2-197b37a307d7>:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Task complete


In [6]:
df

,ABCD Misc,ADS Permission State,Calculated Acres,Containment Date Time,Control Date Time,Daily Acres,Discovery Acres,Dispatch Center ID,Estimated Cost To Date,Final Fire Report Approved Date,...,Unique Fire Identifier,WFDSS Decision Status,Created By System,Modified By System,Is Dispatch Complete,Organizational Assessment,Strategic Decision Publish Date,Created On Date Time,Modified On Date Time,Source
0,,DEFAULT,,,,,,VAVIC,,,...,2021-VAVAF-210004,No Decision,IROC,IROC,0,,,"1/2/2021, 4:53 PM","12/9/2021, 10:29 PM",IRWIN
1,,DEFAULT,,,,324,1,KYKIC,,,...,2021-KYDBF-210099,No Decision,wildcad,wildcad,0,,,"1/23/2021, 6:11 PM","12/9/2021, 6:22 PM",IRWIN
2,,DEFAULT,,,,18,1,SDGPC,,,...,2021-SDSDS-210107,No Decision,wildcad,wildcad,0,,,"2/17/2021, 8:57 PM","12/10/2021, 9:10 PM",IRWIN
3,,DEFAULT,,,,1,1,KYKIC,,,...,2021-KYDBF-210044,No Decision,wildcad,wildcad,0,,,"1/20/2021, 12:35 AM","12/9/2021, 5:52 PM",IRWIN
4,,DEFAULT,,,,1,1,KYKIC,,,...,2021-KYDBF-210043,No Decision,wildcad,wildcad,0,,,"1/20/2021, 12:33 AM","12/9/2021, 5:53 PM",IRWIN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,,DEFAULT,,,,,0.1,CARRCC,,,...,2021-CARRU-172841,No Decision,cfcad,cfcad,0,,,"12/12/2021, 12:33 PM","12/12/2021, 12:33 PM",IRWIN
162,,DEFAULT,,,,,0.1,CARRCC,,,...,2021-CARRU-172844,No Decision,cfcad,cfcad,0,,,"12/12/2021, 12:43 PM","12/12/2021, 12:43 PM",IRWIN
163,,DEFAULT,,,,0.5,0.5,ARAOC,,,...,2021-OKANA-002569,No Decision,wildcad,wildcad,0,,,"12/12/2021, 4:04 PM","12/12/2021, 4:09 PM",IRWIN
164,,DEFAULT,,,,,0.1,CARRCC,,,...,2021-CARRU-172906,No Decision,cfcad,cfcad,0,,,"12/12/2021, 4:52 PM","12/12/2021, 4:52 PM",IRWIN
